### Notes and References
None yet.




In [1]:

#@markdown ###First Cell 
#@markdown *Please paste this cell into any colab notebook for this project.* <br>
#@markdown ***
#@markdown **Contents**: Contents. <br>
#@markdown ***
#@markdown Major Edit History: 
#@markdown - Author, Date: Created as copy of Old Notebook (`Old Title`).
#@markdown ***
#@markdown <br> 
#@markdown Please change `FIELDDAY_DIR` if it is located differently in your drive. This cell will error if `FIELDDAY_DIR` is incorrect.
# mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Change working directory
import os
FIELDDAY_DIR = '/content/drive/My Drive/Field Day' #@param {type:"string"}
INCLUDE_DIR = '/content/drive/My Drive/Field Day/Research and Writing Projects/OGDUtils' #@param (type:"string")
PROJECT_DIR = os.path.join(FIELDDAY_DIR,'Research and Writing Projects/2019 Using Jo Wilder to Predict English Language Skill')
JUPYTER_DIR = os.path.join(PROJECT_DIR,'Colab/')
os.chdir(JUPYTER_DIR)
print(f'---\nCWD: {os.getcwd()}')

#@markdown Change pandas `max_rows` and `max_columns`
import pandas as pd
pd.options.display.max_columns = 100 #@param {type:"integer"}
#pd.options.display.max_columns = 1000
pd.options.display.max_rows = 60 #@param {type:"integer"}

#@markdown *Note: There may be other variables to manually change. Look the "Set Variables" section.*

# import utils
import sys
sys.path.append(INCLUDE_DIR)
#sys.path.append('.')
import utils

Mounted at /content/drive
---
CWD: /content/drive/My Drive/Field Day/Research and Writing Projects/2019 Using Jo Wilder to Predict English Language Skill/Colab


### Imports

In [2]:
# import whole modules
import math
import numpy as np
import pandas as pd
import ipywidgets as widgets
import jowilder_utils as jw_util
import feature_utils as feat_util
import urllib.request
# import submodules
from collections import Counter
from datetime import datetime
from google.colab import files
from io import BytesIO
from math import ceil
from matplotlib import pyplot as plt
from scipy import stats
from Workflow import Workflow
from zipfile import ZipFile
# import items from submodules
from imblearn.pipeline import make_pipeline
# from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.svm import LinearSVC
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import importlib
importlib.reload(feat_util)
importlib.reload(jw_util)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


<module 'jowilder_utils' from '/content/drive/My Drive/Field Day/Research and Writing Projects/OGDUtils/jowilder_utils.py'>

## Import CSV

In [3]:
def open_csv_from_path_with_metadata(csv_fpath, index_col=0):
    metadata = []
    with open(csv_fpath) as f:
        for line in f.readlines():
            if line.startswith('#'):
                metadata.append(line[2:].strip())
            else:
                break;
    df = pd.read_csv(csv_fpath, comment='#', index_col=index_col)
    return df, metadata
print(f"finished execution, {datetime.now()}")

finished execution, 2020-07-10 13:58:40.029027


In [4]:
csv_reldir = "Output Data/JM_202007061210 (June Datasets)" #@param {type:"string"}
csv_fname = "JM_202007061210_withUnreached_obj12-_thresh10_10.csv" #@param {type:"string"}
csv_path = os.path.join(csv_reldir, csv_fname)
print('Open from:', csv_path)
print('Other files in dir:', ''.join([
        f'\n{f}' for f in os.listdir(csv_reldir) if f.endswith('sv')]))

Open from: Output Data/JM_202007061210 (June Datasets)/JM_202007061210_withUnreached_obj12-_thresh10_10.csv
Other files in dir: 
JM_202007061210_onlyNonnative_obj12-_thresh0_0.csv
JM_202007061210_onlyNonnative_obj43-_thresh0_0.csv
JM_202007061210_onlyNonnative_obj76-_thresh0_0.csv
JM_202007061210_withUnreached_onlyNonnative_obj12-_thresh0_0.csv
JM_202007061210_withUnreached_onlyNonnative_obj43-_thresh0_0.csv
JM_202007061210_withUnreached_onlyNonnative_obj76-_thresh0_0.csv
JM_202007061210_obj12-_thresh0_0.csv
JM_202007061210_obj43-_thresh0_0.csv
JM_202007061210_obj76-_thresh0_0.csv
JM_202007061210_withUnreached_obj12-_thresh0_0.csv
JM_202007061210_withUnreached_obj43-_thresh0_0.csv
JM_202007061210_withUnreached_obj76-_thresh0_0.csv
JM_202007061210_onlyNonnative_obj12-_thresh1_1.csv
JM_202007061210_onlyNonnative_obj43-_thresh1_1.csv
JM_202007061210_onlyNonnative_obj76-_thresh1_1.csv
JM_202007061210_withUnreached_onlyNonnative_obj12-_thresh1_1.csv
JM_202007061210_withUnreached_onlyNonnati

In [5]:
# df = pd.read_csv(csv_path, comment='#', index_col=0)
df, meta = open_csv_from_path_with_metadata(csv_path)

print(f"quiz response vars:\n{[col for col in df.columns if col.startswith('R')]}")
# print('\n'.join(meta))
df.describe()

quiz response vars:
['R0_quiz_response', 'R1_quiz_response', 'R2_quiz_response', 'R1_quiz_response_bin', 'R1_quiz_response_0v12', 'R1_quiz_response_01v2', 'R1_quiz_response_bin_x', 'R2_quiz_response_bin', 'R2_quiz_response_bin0v123', 'R2_quiz_response_bin01v23', 'R2_quiz_response_bin012v3', 'R2_quiz_response_bin_x']


,Q0_A1_time,Q0_A2_time,Q0_A3_time,Q0_num_guesses,Q10_A1_time,Q10_A2_time,Q10_A3_time,Q10_num_guesses,Q11_A1_time,Q11_A2_time,Q11_A3_time,Q11_num_guesses,Q12_A1_time,Q12_num_guesses,Q13_A1_time,Q13_A2_time,Q13_A3_time,Q13_num_guesses,Q14_A1_time,Q14_num_guesses,Q15_A1_time,Q15_A2_time,Q15_A3_time,Q15_num_guesses,Q16_A1_time,Q16_num_guesses,Q17_A1_time,Q17_num_guesses,Q18_A1_time,Q18_num_guesses,Q1_A1_time,Q1_num_guesses,Q2_A1_time,Q2_num_guesses,Q3_A1_time,Q3_A2_time,Q3_num_guesses,Q4_A1_time,Q4_A2_time,Q4_num_guesses,Q5_A1_time,Q5_A2_time,Q5_A3_time,Q5_num_guesses,Q6_A1_time,Q6_A2_time,Q6_num_guesses,Q7_A1_time,Q7_A2_time,Q7_A3_time,...,obj63_onext_int_other,obj64_onext_int_other,obj65_onext_int_other,obj66_onext_int_other,obj67_onext_int_other,obj68_onext_int_other,obj70_onext_int_other,obj72_onext_int_other,obj73_onext_int_other,obj74_onext_int_other,obj76_onext_int_other,obj77_onext_int_other,Q0_A2_other,Q1_A2_other,Q2_A2_other,Q3_A2_other,Q4_A2_other,Q5_A2_other,Q6_A2_other,Q7_A2_other,Q8_A2_other,Q9_A2_other,Q10_A2_other,Q11_A2_other,Q12_A2_other,Q13_A2_other,Q14_A2_other,Q15_A2_other,Q16_A2_other,Q17_A2_other,Q18_A2_other,Q0_A3_other,Q1_A3_other,Q2_A3_other,Q3_A3_other,Q4_A3_other,Q5_A3_other,Q6_A3_other,Q7_A3_other,Q8_A3_other,Q9_A3_other,Q10_A3_other,Q11_A3_other,Q12_A3_other,Q13_A3_other,Q14_A3_other,Q15_A3_other,Q16_A3_other,Q17_A3_other,Q18_A3_other
count,21126.000000,9077.000000,5052.000000,21126.000000,11962.000000,6842.000000,4923.000000,11962.000000,11712.000000,4803.000000,3104.000000,11712.000000,11566.000000,11566.000000,11499.000000,9093.000000,5028.000000,11499.000000,6882.000000,6882.000000,6851.000000,3521.000000,2150.000000,6851.000000,6790.000000,6790.000000,6767.000000,6767.000000,6654.000000,6654.000000,21126.000000,21126.000000,6697.000000,6697.000000,14429.00000,2199.000000,14429.000000,12644.000000,2919.000000,12644.000000,12566.000000,6498.000000,5005.000000,12566.000000,12420.000000,3334.000000,12420.000000,12331.000000,3826.000000,2181.000000,...,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.00000,21127.000000,21127.00000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000,21127.000000
mean,8.676775,6.592464,6.687986,1.830351,8.657286,7.539246,7.468369,4.462715,5.330300,6.629955,8.219269,3.037654,6.276525,1.451409,10.620650,10.062016,7.769918,7.475172,16.675076,1.890729,7.990143,4.948101,4.881257,2.977084,5.106573,1.629750,4.432538,1.978425,4.285935,1.170123,6.329792,1.074316,5.258113,1.170673,6.42416,6.054703,1.246725,5.961861,5.441292,2.153828,11.379834,6.670744,6.324972,3.360497,8.012134,5.391651,1.973188,5.977062,5.108451,5.313885,...,0.000663,0.022341,0.058645,0.011313,0.096512,0.054149,0.090406,0.086808,0.098642,0.013017,0.057888,0.002982,0.006106,0.053060,0.029110,0.104085,0.138164,0.307569,0.157808,0.181095,0.068159,0.180764,0.323851,0.227339,0.083022,0.238747,0.090216,0.166659,0.086714,0.098547,0.012969,0.079235,0.012496,0.012117,0.034742,0.071993,0.236901,0.09547,0.103233,0.06797,0.103138,0.233019,0.146921,0.042884,0.237989,0.050362,0.101766,0.048847,0.017986,0.008141
std,11.496114,14.299231,16.466244,1.371528,9.769718,10.539381,7.841404,6.647224,11.050982,18.475492,14.086160,4.962918,10.995400,1.741490,18.969473,20.529091,12.956206,11.459283,19.026921,2.397858,15.784165,5.225798,20.510934,4.446197,10.853368,1.822398,6.433612,6.042035,18.105659,1.391036,8.150720,0.403025,11.502181,0.840997,8.23327,5.698537,0.791102,14.390364,5.832179,3.690787,12.371644,7.064468,9.294848,3.921240,14.158743,

# Detect optimal features

Set up data:

In [6]:
# next, drop outcome columns (i.e. survey responses).
# drops = ['R0_quiz_response', 'R1_quiz_response', 'R1_quiz_response_bin', 'R2_quiz_response', 'R2_quiz_response_bin']
## To remove any other classes of features, uncomment the appropriate line below.
# drops += [elem for elem in feat_detect_data.columns if elem.startswith("int")]
# drops += [elem for elem in feat_detect_data.columns if elem.startswith("lvl")]
# drops += [elem for elem in feat_detect_data.columns if elem.startswith("obj")]
# drops += [elem for elem in feat_detect_data.columns if elem.startswith("Q")]
# feat_detect_data = numeric_df.drop(drops, axis=1)
# print(f"dropping features: {drops}, reduces from {len(numeric_df.columns)} to {len(feat_detect_data.columns)} features.")
transformer, numeric_df = jw_util.get_preprocessor(df)
feat_detect_data_np = transformer.fit_transform(numeric_df)
feat_detect_data = pd.DataFrame(feat_detect_data_np, index=numeric_df.index, columns=numeric_df.columns)

# print(numeric_df.iloc[:,0].head(10))
# print(feat_detect_data_np[:10, 0])
# print(feat_detect_data.iloc[:,0].head(10))
# print(numeric_df.columns[0])
# print(feat_detect_data.columns[0])

# y_keys = ["y0", "y1_bin", "y2_bin", "y1_bin_x", "y2_bin_x"]
y_keys = ["y1_bin_01v2", "y1_bin_0v12", "y2_bin_01v23", "y2_bin_0v123", "y2_bin_012v3"]
predictors = jw_util.get_ys(df)
# bad_indices_R1_x = predictors['y1_bin_x'].index[predictors['y1_bin_x'].isna()]
# bad_indices_R2_x = predictors['y2_bin_x'].index[predictors['y2_bin_x'].isna()]

In [7]:
SELECTION_MODEL_TYPE = "SVM" # "Lasso"
WITH_UNDERSAMPLING = True

if SELECTION_MODEL_TYPE == "Lasso":
  alpha_val = 0.1
  pre_test_models = {key:Lasso(alpha=alpha_val, max_iter=1000) for key in y_keys}
  meta.append(f"Feature Selection: Performed with Lasso, alpha={alpha_val}")
elif SELECTION_MODEL_TYPE == "SVM":
  C_vals = {'y1_bin_01v2':0.0031, 'y1_bin_0v12':0.007, 'y2_bin_0v123':0.0025, 'y2_bin_01v23':0.004, 'y2_bin_012v3':0.01}
  pre_test_models = {key:LinearSVC(penalty='l1', C=C_vals[key], dual=False) for key in y_keys}
if WITH_UNDERSAMPLING:
  # count pre-sampling
  sampler = RandomUnderSampler(random_state=37)
  counters = [str(Counter(predictors[key])) for key in y_keys]
  print(f"Pre-resampling distributions: {', '.join(counters)}")

  # do resampling
  # data_R0, predict_R0_final = sampler.fit_resample(feat_detect_data, predictors['y0'])
  data_per_predictor = {}
  final_predictors = {}
  for key in y_keys:
    next_data, next_predictor = sampler.fit_resample(feat_detect_data, predictors[key])
    data_per_predictor[key] = next_data
    final_predictors[key] = next_predictor
  # data_R1_x, predict_R1_final_x = sampler.fit_resample(feat_detect_data.drop(bad_indices_R1_x), predictors['y1_bin_x'].drop(bad_indices_R1_x))
  # data_R2_x, predict_R2_final_x = sampler.fit_resample(feat_detect_data.drop(bad_indices_R2_x), predictors['y2_bin_x'].drop(bad_indices_R2_x))
  # data_per_predictor = {'y0':data_R0, 'y1_bin':data_R1, 'y2_bin':data_R2, 'y1_bin_x':data_R1_x, 'y2_bin_x':data_R2_x}
  # final_predictors = {'y0':predict_R0_final, 'y1_bin':predict_R1_final, 'y2_bin':predict_R2_final, 'y1_bin_x':predict_R1_final_x, 'y2_bin_x':predict_R2_final_x}

  # count post-sampling
  meta.append("Feature Selection: Performed with Undersampling.")
  counters = [str(Counter(final_predictors[key])) for key in y_keys]
  msg = f"Post-resampling distributions: {', '.join(counters)}"
  print(msg)
  meta.append(f"Feature Selection: {msg}")
else:
  data_per_predictor = {key:feat_detect_data for key in y_keys}
  final_predictors = predictors
  # data_per_predictor = {'y0':feat_detect_data, 'y1_bin':feat_detect_data, 'y2_bin':feat_detect_data,
  #                       'y1_bin_x':feat_detect_data.drop(bad_indices_R1_x),
  #                       'y2_bin_x':feat_detect_data.drop(bad_indices_R1_x)}
  # final_predictors = {'y0':predictors['y0'], 'y1_bin':predictors['y1_bin'], 'y2_bin':predictors['y2_bin'],
  #                     'y1_bin_x':predictors['y1_bin_x'].drop(bad_indices_R1_x),
  #                     'y2_bin_x':predictors['y2_bin_x'].drop(bad_indices_R2_x)}

  # count final predictors
  meta.append("Feature Selection: No Undersampling.")
  counters = [str(Counter(final_predictors[key])) for key in y_keys]
  msg = f"Label distributions: {', '.join(counters)}"
  print(msg)
  meta.append(f"Feature Selection: {msg}")
print(f"finished execution, {datetime.now()}")

Pre-resampling distributions: Counter({1.0: 15305, 0.0: 5822}), Counter({1.0: 19780, 0.0: 1347}), Counter({0.0: 17174, 1.0: 3953}), Counter({0.0: 12042, 1.0: 9085}), Counter({0.0: 20158, 1.0: 969})


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

Post-resampling distributions: Counter({0.0: 5822, 1.0: 5822}), Counter({0.0: 1347, 1.0: 1347}), Counter({0.0: 3953, 1.0: 3953}), Counter({0.0: 9085, 1.0: 9085}), Counter({0.0: 969, 1.0: 969})
finished execution, 2020-07-10 13:59:03.617389


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Generate Models:

In [8]:
pre_test_results = {key:pre_test_models[key].fit(data_per_predictor[key], final_predictors[key]) for key in y_keys}
if SELECTION_MODEL_TYPE == "SVM":
  print(f"Finished running models with C={C_vals}")
else:
  print(f"Finished running models with alpha={alpha_val}")

selectors = {key:SelectFromModel(pre_test_results[key], prefit=True) for key in y_keys}
print(f"finished execution, {datetime.now()}")

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Finished running models with C={'y1_bin_01v2': 0.0031, 'y1_bin_0v12': 0.007, 'y2_bin_0v123': 0.0025, 'y2_bin_01v23': 0.004, 'y2_bin_012v3': 0.01}
finished execution, 2020-07-10 13:59:12.613144


Calculate importance values, and count selected features:

In [9]:
raw_importances = {key:np.abs(pre_test_results[key].coef_) for key in y_keys}
for key in y_keys:
  next_R = raw_importances[key][0]
  print(f"greatest importance {key}: {max(next_R)}")

feat_indices_by_selector = {key:selectors[key].get_support(indices=True) for key in y_keys}
features_by_selector = {key:feat_detect_data.columns[feat_indices_by_selector[key]] for key in y_keys}
importances_by_selector = {key:raw_importances[key][0][feat_indices_by_selector[key]] for key in y_keys}
# importances_by_selector = [[1.0]*len(features_by_selector[i]) for i in range(len(selectors))]

metas = {key:meta.copy() for key in y_keys} # make separate copies for each selector.
for key in y_keys:
  msg = f"Number of features for predicting {key}_quiz_response: {len(feat_indices_by_selector[key])}"
  print(msg) 
  metas[key].append(f"Feature Selection: Performed with SVM, C={C_vals[key]}")
  metas[key].append(f"Feature Selection: {msg}")
  metas[key].append(f"Feature Selection: Selected {features_by_selector[key]}")
# *** Handle Common Features ***
# simple lambda expression to check whether an element is in all lists from a list of lists
# in_index_sets = lambda elem, index_sets : sum([1 if elem in index_set else 0 for index_set in index_sets]) == len(index_sets)
# common_feature_indices = [i for i in feature_indices_by_selector[y_keys[0]] if in_index_sets(i, feature_indices_by_selector[1:])]
# print(f"Number of common features: {len(common_feature_indices)}")

greatest importance y1_bin_01v2: 0.07323710668525678
greatest importance y1_bin_0v12: 0.05339993574424194
greatest importance y2_bin_01v23: 0.04047528468127484
greatest importance y2_bin_0v123: 0.04928942913257778
greatest importance y2_bin_012v3: 0.10429899247150735
Number of features for predicting y1_bin_01v2_quiz_response: 112
Number of features for predicting y1_bin_0v12_quiz_response: 94
Number of features for predicting y2_bin_01v23_quiz_response: 107
Number of features for predicting y2_bin_0v123_quiz_response: 106
Number of features for predicting y2_bin_012v3_quiz_response: 124


Set up chart functions:

In [10]:
# CHART_SIZE_FACTOR = 10 #@param {type:"integer"}
# def FeatureSuggestionPlot(features, importances, title):
#   fig = plt.figure(figsize=[6.4, len(importances)*(CHART_SIZE_FACTOR/10)/4])
#   ax = fig.subplots()
#   y_pos = np.arange(len(features))
#   ax.barh(y_pos, importances, align='center')
#   ax.set_yticks(y_pos)
#   ax.set_yticklabels(features)
#   ax.set_xlabel("Importance")
#   ax.set_title(title)
#   fig.show()
# print(f"finished execution, {datetime.now()}")

**Feature Suggestions List**:

In [11]:
# Now that we've got a bunch of lists ready, we just use them to make graphs
# for i in range(len(selectors)):
#   FeatureSuggestionPlot(features_by_selector[i], importances_by_selector[i], title=f"R{i}_quiz_response")
#   # FeatureSuggestionPlot(features_by_selector[i], [1.0]*len(features_by_selector[i]), title=f"R{i}_quiz_response")

# if len(common_feature_indices) > 0:
#   common_features = feat_detect_data.columns[common_feature_indices]
#   common_importances = [[importances_by_selector[j][i] for j in range(len(selectors))] for i in range(len(common_features)) ]
#   mean = lambda vals : sum(vals) / len(vals)
#   average_common_importances = [mean(importance_list) for importance_list in common_importances]
#   # average_common_importances = [1.0]*len(common_features)
#   print(f"Greatest average importance for common features: {max(average_common_importances)}")
#   FeatureSuggestionPlot(common_features, average_common_importances, title=f"Average Importance per Common Feature")

### Select final features

In [12]:
# feat_util.get_range_feats_and_range(level_features_widget)

In [13]:
# if options:# and options.lvlfeats is not None and options.lvlrange is not None: 
#     aggregate_df, aggregate_meta = feat_util.describe_lvl_feats(df, options.lvlfeats, options.lvlrange)
# else:
#     range_feats_and_range = feat_util.get_range_feats_and_range(level_features_widget)
#     aggregate_df, aggregate_meta = feat_util.des(range_feats_and_range, feat_util.jw_cc_max)
# # if not options:# and options.finalfeats is not None:
#     final_features_widget = feat_util.get_feat_selection(aggregate_df, session_prefix="sess", max_lvl=23, cc_prefix_max_list=feat_util.jw_cc_max)
#     display(final_features_widget)
# print(*aggregate_meta,sep='\n')

### Output final columns

In [10]:
# if options: #None not in [options, options.finalfeats]:
#     reduced_df, reduced_meta = feat_util.reduce_feats(aggregate_df, options.finalfeats)
# else:
#     final_feats = feat_util.get_selected_feature_list(final_features_widget, session_prefix="sess")
#     reduced_df, reduced_meta = feat_util.reduce_feats(aggregate_df, final_feats)
reduced_dfs_by_selector = {key:feat_detect_data[features_by_selector[key]].copy() for key in y_keys}
# reduced_dfs_by_selector[0].loc[:,"R0_quiz_response"] = predictors['y0']
reduced_dfs_by_selector['y1_bin_01v2'].loc[:,"R1_quiz_response_01v2"] = predictors['y1_bin_01v2']
reduced_dfs_by_selector['y1_bin_0v12'].loc[:,"R1_quiz_response_0v12"] = predictors['y1_bin_0v12']
reduced_dfs_by_selector['y2_bin_0v123'].loc[:,"R2_quiz_response_0v123"] = predictors['y2_bin_0v123']
reduced_dfs_by_selector['y2_bin_01v23'].loc[:,"R2_quiz_response_01v23"] = predictors['y2_bin_01v23']
reduced_dfs_by_selector['y2_bin_012v3'].loc[:,"R2_quiz_response_012v3"] = predictors['y2_bin_012v3']
# reduced_dfs_by_selector['y1_bin_x'].loc[:,"R1_quiz_response_bin_x"] = predictors['y1_bin_x']
# reduced_dfs_by_selector['y2_bin_x'].loc[:,"R2_quiz_response_bin_x"] = predictors['y2_bin_x']
# for meta in metas:
  # print('\n'.join(meta))
  # print("***\n\n\n")
# take a quick look at the first set of selected features
print(features_by_selector['y1_bin_01v2'])
# reduced_dfs_by_selector['y1_bin_01v2']

Index(['Q1_num_guesses', 'Q4_num_guesses', 'int118_inum_enc',
       'int11_ifirst_enc_avg_wps', 'int11_ifirst_enc_duration',
       'int127_inum_enc', 'int132_inum_enc', 'int13_ifirst_enc_avg_tbps',
       'int16_ifirst_enc_duration', 'int16_ifirst_enc_words_read',
       ...
       'sess_first_enc_avg_tbps', 'Q5_A1_I', 'obj8_onext_int_103.0',
       'obj14_onext_int_114.0', 'obj18_onext_int_127.0',
       'obj32_onext_int_33.0', 'obj34_onext_int_35.0', 'obj46_onext_int_92.0',
       'obj77_onext_int_nan', 'Q13_A2_N'],
      dtype='object', length=112)


In [11]:
if "R1_quiz_response_01v2" in reduced_dfs_by_selector['y1_bin_01v2'].columns:
  print(f"We got it! {reduced_dfs_by_selector['y1_bin_01v2']['R1_quiz_response_01v2']}")
else:
  print("Missing R1_quiz_response_bin!")
# reduced_dfs_by_selector['y1_bin_01v2'].describe()

We got it! sessionID
19110008122038204    0.0
19110008501053136    1.0
19110009114094480    1.0
19110009240104090    1.0
19110011340239490    1.0
                    ... 
20050621364727810    1.0
20050622050615504    1.0
20050622173866292    1.0
20050623091682920    1.0
20050623142103304    1.0
Name: R1_quiz_response_01v2, Length: 21127, dtype: category
Categories (2, float64): [0.0, 1.0]


In [12]:
final_dfs_by_selector = {}
final_metas_by_selector = metas
for key in y_keys:
  final_df, outlier_meta = (reduced_dfs_by_selector[key], final_metas_by_selector[key]) # feat_util.reduce_outliers(reduced_dfs_by_selector[i], 3)
  # outlier_meta.append(str(features_by_selector[i]))
  final_dfs_by_selector[key] = final_df
  # final_metas_by_selector[i] += outlier_meta
# final_meta = import_meta + filter_meta + new_feat_meta + aggregate_meta + reduced_meta + outlier_meta
# print(*final_metas_by_selector['y1_bin'], sep='\n')

for key in y_keys:
  dec_part = str(C_vals[key] - int(C_vals[key]))[2:]
  # suffix = f"_selected_for_{key}_SVM_C{int(C_val)}-{dec_part}_with_undersampling.csv"
  suffix = f"_selected_for_{key}_SVM_customC_with_undersampling.csv"
  feat_util.save_csv_and_meta(final_dfs_by_selector[key], final_metas_by_selector[key], "Output Data/FSelected", csv_fname[0:-4] + suffix)
  print(f"Output file {csv_fname[0:-4] + suffix}")

Output file JM_202007061210_withUnreached_obj12-_thresh10_10_selected_for_y1_bin_01v2_SVM_customC_with_undersampling.csv
Output file JM_202007061210_withUnreached_obj12-_thresh10_10_selected_for_y1_bin_0v12_SVM_customC_with_undersampling.csv
Output file JM_202007061210_withUnreached_obj12-_thresh10_10_selected_for_y2_bin_01v23_SVM_customC_with_undersampling.csv
Output file JM_202007061210_withUnreached_obj12-_thresh10_10_selected_for_y2_bin_0v123_SVM_customC_with_undersampling.csv
Output file JM_202007061210_withUnreached_obj12-_thresh10_10_selected_for_y2_bin_012v3_SVM_customC_with_undersampling.csv


In [17]:
print(final_metas_by_selector['y1_bin_01v2'])
# print(final_metas_by_selector[1])
# print(final_metas_by_selector[2])
feat_util.print_options(final_metas_by_selector['y1_bin_01v2'])
# feat_util.print_options(final_metas_by_selector[1])
# feat_util.print_options(final_metas_by_selector[2])

['Metadata:', 'Import from fhttps://opengamedata.fielddaylab.wisc.edu/data/JOWILDER/JOWILDER_20191201_to_20191231_ad4c5fa_proc.zip', 'Import from fhttps://opengamedata.fielddaylab.wisc.edu/data/JOWILDER/JOWILDER_20200101_to_20200131_ad4c5fa_proc.zip', 'Import from fhttps://opengamedata.fielddaylab.wisc.edu/data/JOWILDER/JOWILDER_20200201_to_20200229_ad4c5fa_proc.zip', 'Import from fhttps://opengamedata.fielddaylab.wisc.edu/data/JOWILDER/JOWILDER_20200301_to_20200331_ad4c5fa_proc.zip', 'Import from fhttps://opengamedata.fielddaylab.wisc.edu/data/JOWILDER/JOWILDER_20200401_to_20200430_ad4c5fa_proc.zip', 'Import from fhttps://opengamedata.fielddaylab.wisc.edu/data/JOWILDER/JOWILDER_20200501_to_20200531_10529b0_proc.zip', 'Import from fhttps://opengamedata.fielddaylab.wisc.edu/data/JOWILDER/JOWILDER_20200601_to_20200630_f1c3c64_proc.zip', 'Added feat: times_in_objectives = sum(obj{i}_otime_to_next_obj for i in range(80))', 'Added feat: levels_played = (max_level-start_level+1)', 'Added fea

In [18]:
# Workflow.Histogram(final_df, title='Feature Histogram')

In [19]:
# Workflow.Correlations(final_df)

In [20]:
# colors = {i: v for i, v in enumerate(plt.cm.get_cmap('tab10').colors)}
# Workflow.scatter(final_df, [0]*len(final_df), color_dict=colors, title='Scatter')